# Hyperspectral Image Classification with Hybrid Hypergraph Attention Network

In [ ]:
# Import necessary libraries
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import torch
import torch.optim as optim
import torch.nn as nn
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from skimage.segmentation import slic

from src.models.hybrid_hypergraph_attention import build_model, create_hypergraph_adjacency_matrix
from src.utils.data_loader import load_data
from src.utils.preprocessing import normalize_data, apply_pca, segment_image, preprocess_data
from src.utils.evaluation import evaluate_model

In [ ]:
# Load dataset
print("Loading dataset...")
data, labels = load_data('data/IP/IP.mat')
print("Dataset loaded. Data shape:", data.shape, "Labels shape:", labels.shape)

In [ ]:
# Data visualization
print("Visualizing data...")
plt.figure(figsize=(10, 6))
sns.countplot(labels)
plt.title('Class Distribution in the Dataset')
plt.xlabel('Class')
plt.ylabel('Frequency')
plt.show()

In [ ]:
# Normalize data
print("Normalizing data...")
data_normalized = normalize_data(data)

In [ ]:
# PCA for dimensionality reduction
print("Applying PCA...")
data_pca = apply_pca(data_normalized, n_components=30)

In [ ]:
# SLIC for superpixel segmentation
print("Segmenting data using SLIC...")
segments = segment_image(data_pca, n_segments=100)

In [ ]:
# Visualize a sample of the segmented data
print("Visualizing segmented data...")
plt.figure(figsize=(10, 6))
plt.imshow(segments, cmap='gray')
plt.title('Sample Segmented Image')
plt.axis('off')
plt.show()

In [ ]:
# Split the dataset into training and testing sets
print("Splitting dataset into training and testing sets...")
X_train, X_test, y_train, y_test = train_test_split(segments, labels, test_size=0.2, random_state=42)
print("Dataset split. Training data shape:", X_train.shape, "Testing data shape:", X_test.shape)

In [ ]:
# Create hypergraph adjacency matrix for training data
print("Creating hypergraph adjacency matrix for training data...")
adjacency_matrix_train = create_hypergraph_adjacency_matrix(X_train, n_superpixels=100)

In [ ]:
# Create hypergraph adjacency matrix for testing data
print("Creating hypergraph adjacency matrix for testing data...")
adjacency_matrix_test = create_hypergraph_adjacency_matrix(X_test, n_superpixels=100)

In [ ]:
# Build model
print("Building the model...")
model = build_model(input_dim=30, hidden_dim=64, output_dim=len(np.unique(labels)))
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()
print("Model built.")

In [ ]:
# Train model
print("Training the model...")
model.train()
num_epochs = 100
train_losses = []

for epoch in range(num_epochs):
    optimizer.zero_grad()
    outputs = model(torch.tensor(X_train, dtype=torch.float32), adjacency_matrix_train)
    loss = criterion(outputs, torch.tensor(y_train, dtype=torch.long))
    loss.backward()
    optimizer.step()
    train_losses.append(loss.item())
    print(f'Epoch {epoch+1}/{num_epochs}, Loss: {loss.item()}')

In [ ]:
# Plot training loss
print("Plotting training loss...")
plt.figure(figsize=(10, 6))
plt.plot(train_losses, label='Training Loss')
plt.title('Training Loss over Epochs')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
# Evaluate model
print("Evaluating the model...")
oa, aa, kappa = evaluate_model(model, X_test, y_test, adjacency_matrix_test)
print(f'Evaluation results - OA: {oa}, AA: {aa}, Kappa: {kappa}')

In [ ]:
# Detailed classification report
print("Detailed classification report...")
from sklearn.metrics import classification_report
model.eval()
with torch.no_grad():
    outputs = model(torch.tensor(X_test, dtype=torch.float32), adjacency_matrix_test)
    _, predicted = torch.max(outputs.data, 1)
    predicted = predicted.numpy()
    y_test = np.array(y_test)
    print(classification_report(y_test, predicted))

In [ ]:
# Hyperparameter tuning example (Learning Rate)
print("Hyperparameter tuning example (Learning Rate)...")
learning_rates = [0.001, 0.01, 0.1]
tuning_results = []

for lr in learning_rates:
    model = build_model(input_dim=30, hidden_dim=64, output_dim=len(np.unique(labels)))
    optimizer = optim.Adam(model.parameters(), lr=lr)
    model.train()
    for epoch in range(num_epochs):
        optimizer.zero_grad()
        outputs = model(torch.tensor(X_train, dtype=torch.float32), adjacency_matrix_train)
        loss = criterion(outputs, torch.tensor(y_train, dtype=torch.long))
        loss.backward()
        optimizer.step()
    
    oa, aa, kappa = evaluate_model(model, X_test, y_test, adjacency_matrix_test)
    tuning_results.append((lr, oa, aa, kappa))

In [ ]:
# Plotting hyperparameter tuning results
print("Plotting hyperparameter tuning results...")
learning_rates, oa_scores, aa_scores, kappa_scores = zip(*tuning_results)

plt.figure(figsize=(10, 6))
plt.plot(learning_rates, oa_scores, label='Overall Accuracy')
plt.plot(learning_rates, aa_scores, label='Average Accuracy')
plt.plot(learning_rates, kappa_scores, label='Kappa')
plt.xscale('log')
plt.xlabel('Learning Rate')
plt.ylabel('Score')
plt.title('Hyperparameter Tuning (Learning Rate)')
plt.legend()
plt.show()

In [ ]:
# Ablation study (Removing PCA)
print("Ablation study (Removing PCA)...")
segments_no_pca = segment_image(normalize_data(data), n_segments=100)
X_train_no_pca, X_test_no_pca, y_train_no_pca, y_test_no_pca = train_test_split(segments_no_pca, labels, test_size=0.2, random_state=42)
adjacency_matrix_train_no_pca = create_hypergraph_adjacency_matrix(X_train_no_pca, n_superpixels=100)
adjacency_matrix_test_no_pca = create_hypergraph_adjacency_matrix(X_test_no_pca, n_superpixels=100)

model_no_pca = build_model(input_dim=data.shape[-1], hidden_dim=64, output_dim=len(np.unique(labels)))
optimizer_no_pca = optim.Adam(model_no_pca.parameters(), lr=0.001)
model_no_pca.train()

for epoch in range(num_epochs):
    optimizer_no_pca.zero_grad()
    outputs = model_no_pca(torch.tensor(X_train_no_pca, dtype=torch.float32), adjacency_matrix_train_no_pca)
    loss = criterion(outputs, torch.tensor(y_train_no_pca, dtype=torch.long))
    loss.backward()
    optimizer_no_pca.step()

oa_no_pca, aa_no_pca, kappa_no_pca = evaluate_model(model_no_pca, X_test_no_pca, y_test_no_pca, adjacency_matrix_test_no_pca)
print(f'Ablation study results (Without PCA) - OA: {oa_no_pca}, AA: {aa_no_pca}, Kappa: {kappa_no_pca}')